# Практикум 1. Поиск структуры в банке. Визуализация структуры. Отображения.

## Импортируем все необходимое

In [1]:
import nglview as nv
import requests         # Этот стандартный модуль позволяет посылать запросы к веб-ресурсам
import json             # Этот стандартный модуль позволяет легко работать с текстовым форматом json

## Задание 1. Подбор подходящей записи из базы PDB

Допустим, мне достался следующий вариант:

*   для сильных духом 3
*   Ключевые слова: erythromycin
*   Где они должны быть: В заголовке (title)
*   Какое разрешение: не хуже 3 ангстрем
*   Сколько лигандов должно быть в записи: не меньше 1
*   Дата публикации записи: с 2010-01-01 по 2015-01-01

In [2]:
from rcsbsearchapi import AttributeQuery


title = AttributeQuery(
    attribute="struct.title",
    operator="contains_words",
    value="erythromycin",
)
resolution = AttributeQuery(
    attribute="rcsb_entry_info.resolution_combined",
    operator="less_or_equal",
    value=3.0
)
ligands = AttributeQuery(
    attribute="rcsb_entry_info.nonpolymer_entity_count",
    operator="greater",
    value=0
)
timerange = AttributeQuery(
    attribute="rcsb_accession_info.initial_release_date",
    operator="range",
    value={
            "from": "2010-01-01",
            "to": "2015-01-01",
        }
)


results = title & resolution & ligands & timerange
best_match = list(results())[0]
print(best_match)

4V7X


## Задание 2. Что нашлось? 4V7X

Thermus thermophilus ribosome complexed with erythromycin.

>Какое отношение она имеет к ключевым словам запроса?

так как запрос был указан как "в названии есть erythromycin" - нашлась рибосома в комплексе с эритромицином

>Какова роль молекулы из запроса в жизни остальных молекул из записи?

роль эритромицина в исследовании в том, чтобы показать схожесть комплексов между видами, 

чтобы объяснить вариацию в моделях докинга этого антибиотика некачественными данными или интерпретациями этих данных в предыдущих исследованиях

## Задание 3. Визуализация

В этом задании от вас ожидается создание кода для трех визуализаций записи из банка данных PDB, которую вы нашли в задании 1 и описали в задании 2.

## Так как в моем варианте задача была "тип отображения: 1,2,8" - Я поставлю описания в эти варианты, чтобы было проще соотносить

In [3]:
from ipywidgets import interact, fixed


representations = [
  'ball+stick',      # шаростержневая модель
  'backbone',        # прямые линии, соединяющие С-альфа атомы
  'cartoon',         # условное отображение вторичной структуры
  'helixorient',     # упрощенное отображение
  'hyperball',       # сглаженная шаростержневая модель
  'licorice',        # стержневая модель (без шаров)
  'line',            # очень тонкая стержневая модель
  'point',           # атомы в виде плоских точек
  'ribbon',          # отображение остова в виде ленты
  'rocket',          # отображение альфа-спиралей как цилиндров
  'rope',            # отображение остова в виде колбасы
  'spacefill',       # атомы в виде шаров соотв. Ван-дер-Ваальсова радиуса
  'surface',         # поверхность
  'trace',           # сглаженная линия, соединяющая С-альфа атомы
  'tube'             # более сдержанный rope
]

task_representation = {
    '1': {
      'description': '2',
      'representations': [
            {
                'type': representations[1],
                'params': {
                    'sele': 'protein',
                    'color': 'element',
                 },
            },
        ],
    },
    '2': {
      'description': '2',
      'representations': [
            {
                'type': representations[2],
                'params': {
                    'sele': 'sidechainAttached',
                    'color': 'hydrophobicity',
                  },
            },
        ],
    },
    '8': {
      'description': '8',
      'representations': [
            {
                'type': representations[8],
                'params': {
                    'sele': 'all',
                    'color': 'residueindex'
                 },
            },
        ],
    },
}


def change_representation(view, example_name):
    view.clear()
    view.set_representations(task_representation[example_name]['representations'])
    view.center()


view = nv.show_pdbid(best_match, default_representation=False)
interact(change_representation, view=fixed(view), example_name=list(task_representation.keys()))
view.center()
view

interactive(children=(Dropdown(description='example_name', options=('1', '2', '8'), value='1'), Output()), _do…

NGLWidget()